# PER / BPR 로 적정 주가 산출 후 현재 주가가 적정 주가보다 20% 이상 낮을 경우 매수

In [1]:
#PER 
#적정 PER -> 낮을수록 좋다. 10이하 6~4정도면 좋음
#5년 평균 PER * EPS = 적정주가 
#적정주가 /1.2 OR 0.8 = 20% 낮은 가격 ->매수 
#적정주가 /현재주가 = 1 -> 현재주가가 적정하다 
#0.96 -> 4퍼센트정도 과대평가 
#1.04 -> 4%정도 낮은 주가 

#PBR
#적정 PBR -> 낮을 수록 좋다. 1이면 현재 적정주가라는 것, 그 이하라면 좋음 
#가장 최근 PBS (PBS는 그간 쌓인 자산이니 가장 최근 값을 보면 된다)
#5년 평균 PBR * BPS = 적정주가 * 0.8 -> 매수 


#미래 주가 예측 
#미래 적정 주가 = 미래의 EPS(애널리스트 추정치) * 적정PER(5년간의 평균)

In [1]:
import urllib.parse
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import time

MARKET_CODE_DICT = {
    
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt',
    'konex': 'konexMkt'
    
}
DOWNLOAD_URL = 'https://kind.krx.co.kr/corpgeneral/corpList.do'
naver_stock = 'https://finance.naver.com/item/main.nhn?code='

#종목코드 6자리 미만시 사용할 함수 
def zeroFill(columnValue):
    
    columnValue = str(columnValue)
    #앞을 0으로 채워라 제로필
    outValue = columnValue.zfill(6)  

    return outValue


#코스피 종목 불러올 함수 설정 
def get_stock_codes(market=None, delisted=False):
    
    params = {'method': 'download'}
    
    if market.lower() in MARKET_CODE_DICT:
        ## marketType 키 추가
        params['marketType'] = MARKET_CODE_DICT[market]
        #print(market.lower()+" market key is exist")
        
    else:
        print("invalid market")
    # make url  key=value & key = value
    params_string = urllib.parse.urlencode(params)
    request_url = DOWNLOAD_URL+"?"+params_string
    df = pd.read_html(request_url)[0]
    df["종목코드"] = df.종목코드.apply(zeroFill)
 
    return df


#종목 테이블에서 종목코드(100개) 가져오는 함수 
def get_code(market):
    
    codeList = []
    stocks = get_stock_codes(market)
    for i in range(100):
        stock_code = stocks.iloc[i]['종목코드']
        codeList.append(stock_code)
        #stock_name = stocks.iloc[0]['회사명']
        #print(stock_code)
        
    return codeList

In [2]:
#주가정보 테이블 생성 함수 
def stock_info(stockCode):
    url = naver_stock + stockCode #종목코드 url에 추가 
    table_df_list = pd.read_html(url, encoding='euc-kr')    # 한글이 깨짐. utf-8도 깨짐. 그래서 'euc-kr'로 설정함  
    table_df = table_df_list[3] #종목 정보가 담겨있는 테이블 추출 
    return table_df

In [12]:
#현재년도의 bps와 per 값 구하해 매입목표가격 정의 
def get_target_price(stockInfo): 
    
    bps_this_year = stockInfo.iloc[11,-2] #bps값 정의
    pbr_this_year = stockInfo.iloc[12,-2] #pbr값 정의
    try:
        just_price = pbr_this_year * bps_this_year #적정가격 도출 
    except :
        just_price = 100000000
    target_price = just_price * 0.8 #적정가격보다 20% 더 저렴한 가격 도출 
    
    return target_price

In [13]:
def get_price(stockCode):
    
    url = naver_stock + stockCode #종목 코드 url에 추가 
    result = requests.get(url)
    html = BeautifulSoup(result.content, "html.parser") #태그에서 현재가격 가지고오기 
    today = html.find("p", {"class":"no_today"}) 
    real_time_tag = today.find("span", {"class":"blind"})
    real_time_price = int(real_time_tag.text.replace(',',"")) #컴마 제거 후 인트형으로 변환 
    
    return real_time_price

In [14]:
def stock_advisor(market):
    codeList = get_code('market')
    for i in range(100):
        print(1)
        stockInfo = stock_info(codeList[i])
        target_price = get_target_price(stockInfo)
        real_time_price = get_price(codeList[i])
        
        result = ""
        if(real_time_price > target_price):
            result = "Y"
        else : 
            result = "N"
            
        pString = "종목코드 : {} / 적정가격 : {} / 현재가격 : {} / 매수여부 {}".format(codeList[i],target_price,real_time_price,result)
        print(pString)
        
    

In [15]:
#stockName = stocks[stocks['회사명'] == 'LG화학'].iloc[0]['종목코드']  종목명으로 종목코드 찾기
#result

In [16]:
stock_advisor("kospi")

invalid market
1
종목코드 : 004840 / 적정가격 : 5342.400000000001 / 현재가격 : 8480 / 매수여부 Y
1
종목코드 : 155660 / 적정가격 : 4563.240000000001 / 현재가격 : 7060 / 매수여부 Y
1
종목코드 : 001250 / 적정가격 : 1996.696 / 현재가격 : 3185 / 매수여부 Y
1
종목코드 : 294870 / 적정가격 : 23349.6 / 현재가격 : 30000 / 매수여부 Y
1
종목코드 : 092220 / 적정가격 : 1856.696 / 현재가격 : 2570 / 매수여부 Y
1
종목코드 : 016380 / 적정가격 : 12278.480000000001 / 현재가격 : 19950 / 매수여부 Y
1
종목코드 : 001390 / 적정가격 : 21584.880000000005 / 현재가격 : 29400 / 매수여부 Y
1
종목코드 : 058860 / 적정가격 : 2391.4240000000004 / 현재가격 : 3290 / 매수여부 Y
1
종목코드 : 011070 / 적정가격 : 164723.032 / 현재가격 : 211000 / 매수여부 Y
1
종목코드 : 066570 / 적정가격 : 119905.408 / 현재가격 : 151500 / 매수여부 Y
1
종목코드 : 037560 / 적정가격 : 3914.4 / 현재가격 : 7890 / 매수여부 Y
1
종목코드 : 010060 / 적정가격 : 97583.04 / 현재가격 : 122500 / 매수여부 Y
1
종목코드 : 096770 / 적정가격 : 175101.50400000002 / 현재가격 : 274500 / 매수여부 Y
1
종목코드 : 011810 / 적정가격 : 4839.535999999999 / 현재가격 : 9020 / 매수여부 Y
1
종목코드 : 024070 / 적정가격 : 80000000.0 / 현재가격 : 4830 / 매수여부 N
1
종목코드 : 011420 / 적정가격 : 1432.32 / 현재가격 : 2395 / 

IndexError: single positional indexer is out-of-bounds

In [43]:
get_target_price('035250')

20238.08

In [60]:
get_price('035250')

28200

In [52]:
a = '28,200'
a = a.replace(',',"")

In [55]:
int(a)

28200

In [57]:
b = 28200

In [58]:
if int(a)==b :
    print(0)
else:
    print(1)

0


In [7]:
url = 'https://finance.naver.com/item/main.nhn?code=035250'
table_df_list = pd.read_html(url, encoding='euc-kr')    # 한글이 깨짐. utf-8도 깨짐. 그래서 'euc-kr'로 설정함  
table_df = table_df_list[3]

In [8]:
eps_2018 = table_df.iloc[9,1]
per_2018 = table_df.iloc[10,1]
bps_2018 = table_df.iloc[11,1]
pbr_2018 = table_df.iloc[12,1]
roe_2018 = table_df.iloc[5,1]

In [9]:
eps_2019 = table_df.iloc[9,2]
per_2019 = table_df.iloc[10,2]
bps_2019 = table_df.iloc[11,2]
pbr_2019 = table_df.iloc[12,2]
roe_2019 = table_df.iloc[5,2]

In [10]:
eps_2021 = table_df.iloc[9,-2]
per_2021 = table_df.iloc[10,-2]
bps_2021 = table_df.iloc[11,-2]
pbr_2021 = table_df.iloc[12,-2]
roe_2021 = table_df.iloc[5,-2]

In [11]:
eps_2020 = table_df.iloc[9,3]
per_2020 = table_df.iloc[10,3]
bps_2020 = table_df.iloc[11,3]
pbr_2020 = table_df.iloc[12,3]
roe_2020 = table_df.iloc[5,3]

In [17]:
eps2018 = eps_2018 * roe_2018 * 100
titrationPrice_2018 = per_2018 * eps_2018
pbr2020 =  pbr_2018 * bps_2018
print(eps2018)
print(titrationPrice_2018)
print(pbr2020)

1166210.0
32011.7
32020.71


In [18]:
eps2019 = eps_2019 * roe_2019 * 100
titrationPrice_2019 = per_2019 * eps_2019
pbr2019 =  pbr_2019 * bps_2019
print(eps2019)
print(titrationPrice_2019)
print(pbr2019)

1432624.0
29590.88
29522.570000000003


In [19]:
eps2020 = eps_2020 * roe_2020 * 100
titrationPrice_2020 = per_2020 * eps_2020
pbr2020 =  pbr_2020 * bps_2020
print(eps2020)
print(titrationPrice_2020)
print(pbr2020)

TypeError: can't multiply sequence by non-int of type 'str'

In [20]:
eps2021 = eps_2021 * roe_2021 * 100
titrationPrice_2021 = per_2021 * eps_2021
pbr2021 = pbr_2021 * bps_2021
print(eps2021)
print(titrationPrice_2021)
print(pbr2021)

93208.0
6436.700000000001
25297.600000000002


In [21]:
#result = stocks[stocks['회사명'] == 'LG화학'].iloc[0]['종목코드']  종목명으로 종목코드 찾기
#result

In [71]:
table_df_list[11]

,업체명,거래량,전일비
0,두산중공업,23400.0,"하향 2,700"
1,신풍제약,75600.0,"상향 14,500"
2,삼성전자,81000.0,하향 100
3,카카오,133500.0,"상향 4,500"
4,HMM,43900.0,상향 150
5,NaN,NaN,NaN
6,대한전선,3105.0,하향 95
7,NAVER,373500.0,"상향 15,000"
8,만도,65200.0,"하향 8,200"
9,쌍방울,1050.0,상향 143


In [46]:
bps

주요재무정보 최근 연간 실적                              최근 분기 실적                    \
    주요재무정보  2018.12  2019.12  2020.12 2021.12(E)  2020.03  2020.06  2020.09   
    주요재무정보   IFRS연결   IFRS연결   IFRS연결     IFRS연결   IFRS연결   IFRS연결   IFRS연결   
11  BPS(원)  12979.0  12746.0  14647.0    16237.0  13108.0  14073.0  14910.0   

                                 
    2020.12  2021.03 2021.06(E)  
     IFRS연결   IFRS연결     IFRS연결  
11  14647.0  15428.0        NaN

In [47]:
pbr

주요재무정보 최근 연간 실적                            최근 분기 실적                  \
    주요재무정보  2018.12 2019.12 2020.12 2021.12(E)  2020.03 2020.06 2020.09   
    주요재무정보   IFRS연결  IFRS연결  IFRS연결     IFRS연결   IFRS연결  IFRS연결  IFRS연결   
12  PBR(배)     1.59    2.41    5.32       7.82     2.37     3.8    4.89   

                               
   2020.12 2021.03 2021.06(E)  
    IFRS연결  IFRS연결     IFRS연결  
12    5.32    6.46        NaN

In [28]:
table_df.loc[table_df.INDEX ==10]

AttributeError: 'DataFrame' object has no attribute 'INDEX'

In [17]:
table_df2 = table_df_list[4]
table_df2

,종목명,카카오*035720,NAVER*035420,아프리카TV*067160,키다리스튜디오*020120,자이언트스텝*289220
0,현재가,126500,362000,110700,15850,42000
1,전일대비,하향 500,"하향 1,000","상향 9,400",하향 100,상향 300
2,등락률,하향 -0.39%,하향 -0.28%,상향 +9.28%,하향 -0.63%,상향 +0.72%
3,시가총액(억),561479,594633,12724,5188,3996
4,외국인비율(%),33.34,56.56,50.40,1.12,.12
5,매출액(억),12580,14991,609,199,70
6,영업이익(억),1575,2888,196,23,-5
7,조정영업이익(억),1575,2888,196,23,-5
8,영업이익증가율(%),5.27,-10.79,10.67,285.57,-319.72
9,당기순이익(억),2399,153145,154,19,-4


In [23]:
table_df3 = table_df_list[4]
table_df3

,종목명,카카오*035720,NAVER*035420,아프리카TV*067160,키다리스튜디오*020120,자이언트스텝*289220
0,현재가,126500,362000,110700,15850,42000
1,전일대비,하향 500,"하향 1,000","상향 9,400",하향 100,상향 300
2,등락률,하향 -0.39%,하향 -0.28%,상향 +9.28%,하향 -0.63%,상향 +0.72%
3,시가총액(억),561479,594633,12724,5188,3996
4,외국인비율(%),33.34,56.56,50.40,1.12,.12
5,매출액(억),12580,14991,609,199,70
6,영업이익(억),1575,2888,196,23,-5
7,조정영업이익(억),1575,2888,196,23,-5
8,영업이익증가율(%),5.27,-10.79,10.67,285.57,-319.72
9,당기순이익(억),2399,153145,154,19,-4


In [33]:
table_df_list[4]

,종목명,카카오*035720,NAVER*035420,아프리카TV*067160,키다리스튜디오*020120,자이언트스텝*289220
0,현재가,126500,362000,110700,15850,42000
1,전일대비,하향 500,"하향 1,000","상향 9,400",하향 100,상향 300
2,등락률,하향 -0.39%,하향 -0.28%,상향 +9.28%,하향 -0.63%,상향 +0.72%
3,시가총액(억),561479,594633,12724,5188,3996
4,외국인비율(%),33.34,56.56,50.40,1.12,.12
5,매출액(억),12580,14991,609,199,70
6,영업이익(억),1575,2888,196,23,-5
7,조정영업이익(억),1575,2888,196,23,-5
8,영업이익증가율(%),5.27,-10.79,10.67,285.57,-319.72
9,당기순이익(억),2399,153145,154,19,-4


In [34]:
table_df_list

[                      0                                            1  \
 0  전일  127,000  127,000  고가  128,500128,500  (상한가  165,000165,000  )   
 1    시가  127,500127,500          저가  126,000126,000  (하한가  89,000  )   
 
                             2  
 0   거래량  1,717,349  1,717,349  
 1  거래대금  218,403  218,403  백만  ,
      매도상위       거래량    매수상위       거개량
 0     NaN       NaN     NaN       NaN
 1  미래에셋대우  267552.0  미래에셋대우  283126.0
 2    한국증권  169383.0    키움증권  188078.0
 3      삼성  163572.0  NH투자증권  184365.0
 4  NH투자증권  145691.0      삼성  176343.0
 5    키움증권  145368.0    한국증권   64184.0
 6  외국계추정합   32555.0  -19521   13034.0,
       날짜        종가        전일비         외국인        기관
 0    NaN       NaN        NaN         NaN       NaN
 1  06/03  126500.0    하향  500      -66691    -29463
 2  06/02  127000.0         보합     -106664    -76154
 3  06/01  127000.0  상향  4,000    +851,809  +151,531
 4  05/31  123000.0  상향  1,000    +514,421   -161659
 5  05/28  122000.0  하향  3,000     -858820  +183